In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet import MobileNet

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
train_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/seg_train/seg_train'
test_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/seg_test/seg_test'

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 15,
                                   shear_range = 0.2,
                                   horizontal_flip = True,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   zoom_range = 0.25)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(train_path,
                                                    batch_size = 64,
                                                    target_size = (224, 224),
                                                    shuffle = True)
test_generator = test_datagen.flow_from_directory(test_path,
                                                  batch_size = 64,
                                                  target_size = (224, 224),
                                                  shuffle = False)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [5]:
from tensorflow.keras import Sequential

baseModel = MobileNet(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
baseModel.trainable = True
model = Sequential()
model.add(baseModel)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation = 'softmax'))

17225924/17225924 [==============================] - 0s 0us/step


In [6]:
adam = Adam(learning_rate = 0.001)

model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs = 5, validation_data = test_generator, batch_size = 64)

Epoch 1/5
 88/220 [===========>..................] - ETA: 49:42 - loss: 0.5792 - accuracy: 0.8093

In [ ]:
preds = model.predict(test_generator)

In [ ]:
out = preds.argmax(axis = 1)
out.shape

In [ ]:
from sklearn.metrics import accuracy_score

dogruluk = accuracy_score(test_generator.labels, out)
print(f"Doğruluk Oranı: {dogruluk: .2f}")